In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document  # 수정된 부분

In [3]:
# Google 스프레드시트 API 인증 범위 설정
scope = ['https://spreadsheets.google.com/feeds']

# 서비스 계정 JSON 키 파일 경로 설정
json_key_path = "key.json"	

# 인증 정보 로드
credential = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
gc = gspread.authorize(credential)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기
sheet_action = doc.worksheet("행위별 Category")
sheet_code = doc.worksheet("상병 Category")

# 데이터 프레임으로 변환하기
df_action = pd.DataFrame(sheet_action.get_all_values())
df_code = pd.DataFrame(sheet_code.get_all_values())

# 첫 번째 행을 컬럼명으로 설정하고 나머지 데이터로 변환
df_action.rename(columns=df_action.iloc[0], inplace=True)
df_action.drop(df_action.index[0], inplace=True)

# 첫 번째 행을 컬럼명으로 설정하고 나머지 데이터로 변환
df_code.rename(columns=df_code.iloc[0], inplace=True)
df_code.drop(df_code.index[0], inplace=True)



In [4]:
df_action

,보험 청구 Category,세부 청구 항목 (키워드),URL (tripletclover.com)
1,1. 청구 기초 및 기본진료,"구내염처치, 단순염증 처치",https://tripletclover.com/%ea%b5%ac%eb%82%b4%e...
2,1. 청구 기초 및 기본진료,기본진료 종류 및 상병,https://tripletclover.com/%ea%b8%b0%eb%b3%b8%e...
3,1. 청구 기초 및 기본진료,기본진료시 항생제 처방 상병,https://tripletclover.com/%ea%b8%b0%eb%b3%b8-%...
4,1. 청구 기초 및 기본진료,상세불명 vs 기타 명시된 상병명 차이,https://tripletclover.com/%ec%83%81%ec%84%b8%e...
5,1. 청구 기초 및 기본진료,전달마취,https://tripletclover.com/%ec%a0%84%eb%8b%ac%e...
...,...,...,...
107,8. 턱관절,악관절 탈구 비관혈적 정복술,https://tripletclover.com/%ec%95%85%ea%b4%80%e...
108,8. 턱관절,악관절고착해소술,https://tripletclover.com/%ec%95%85%ea%b4%80%e...
109,8. 턱관절,"악관절자극요법 (단순, 전기, 복합)",https://tripletclover.com/%ec%95%85%ea%b4%80%e...
110,8. 턱관절,측두하악장애 분석검사,https://tripletclover.com/%ec%b8%a1%eb%91%90%e...


In [24]:
df_action.iloc[2,2]

'https://tripletclover.com/%ea%b8%b0%eb%b3%b8-%ec%a7%84%ec%b0%b0-%ec%b2%98%eb%b0%a9%ec%a0%84-%eb%b0%9c%ea%b8%89-%ed%95%ad%ec%83%9d%ec%a0%9c-%ec%b2%98%eb%b0%a9-%ea%b0%80%eb%8a%a5%ed%95%9c-%ec%a6%9d%ec%83%81-%eb%b0%8f-%ec%83%81/'

In [30]:
df_code['\x08불완전 상병명'].replace("", None).dropna()

1                 치아 발육 및 맹출 장애
24                    매몰치 및 매복치
26                            
37                        치아 우식
44               마모 등 치아 경조직 질환
52                      치수 및 근단
74                   치은염 및 치주질환
100        잇몸 및 무치성 치조융기의 기타 장애
111          치아얼굴이상 [ 부정교합 포함 ]
124            치아 및 지지구조의 기타 장애
130                    턱의 기타 질환
135                      침샘의 질환
140                 구내염 및 관련 병변
144            입술 및 구강점막의 기타 질환
149                  머리의 표재성 손상
151                    머리의 열린상처
163               두개골 및 안면골의 골절
173    머리의 관절 및 인대의 탈구, 염좌 및 긴장
182                     뇌신경의 손상
187                 헤르페스바이러스 감염
189                입 인두의 양성 신생물
191                  위염 및 십이지장염
195                  혀입인두의 선천기형
Name: 불완전 상병명, dtype: object